In [26]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

In [27]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

2.8.0


In [28]:
input_bucket = "sagemaker-us-east2-models"
filename = "output2.h5"
model_version = '2'

In [5]:
!mkdir keras_model
!aws s3 cp s3://$input_bucket/$filename $filename

mkdir: cannot create directory ‘keras_model’: File exists
download: s3://sagemaker-us-east2-models/output2.h5 to ./output2.h5


In [6]:
import os
import tensorflow as tf
import tensorflow.keras as keras
from keras import backend as K

In [7]:
K.set_learning_phase(0)

/opt/conda/lib/python3.7/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [8]:
loaded_model = keras.models.load_model(filename)

In [9]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed 
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at path defined by export_dir variable
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

# Save the meta graph and variables
builder.add_meta_graph_and_variables(
    sess=K.get_session(), tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/2/saved_model.pb


b'export/Servo/2/saved_model.pb'

In [10]:
model_path = export_dir + '/'
!saved_model_cli show --all --dir {model_path}

2022-09-20 16:46:06.374514: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-20 16:46:06.374553: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: dense_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['score'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: dense_5/Sigmoid:0
  Method name is: tensorflow/serving/predict


In [11]:
import tarfile
model_archive = 'model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

# upload model artifacts to S3
model_data = sess.upload_data(path=model_archive, key_prefix='model'+model_version)

In [12]:
from sagemaker.tensorflow import TensorFlow

TensorFlow(
    entry_point="script.py",
    framework_version="2.2.0",  # now required
    py_version="py37",  # now required
    role="my-role",
    instance_type="ml.m5.xlarge",
    instance_count=1,
)


In [13]:
from sagemaker.tensorflow.serving import Model

# Select which type of SageMaker EC2 instance to deploy the model on  
instance_type = 'ml.c5.xlarge' 

# Instantiate the SageMaker TensorFlow serving model  
sm_model = Model(model_data=model_data, 
                 framework_version=tf_framework_version,
                 role=role)

uncompiled_predictor = sm_model.deploy(initial_instance_count=1, instance_type=instance_type) 

The class sagemaker.tensorflow.serving.Model has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [14]:
import numpy as np

In [15]:
# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 4)

deployed_model_preds = uncompiled_predictor.predict(data)

# Also get original model predictions for the same data 
original_model_preds = loaded_model.predict(data)

try: 
    deployed_model_preds == original_model_preds
    assert True
except Exception:
    print("Looks like your deployed model doesn't work exactly like your original model!")   

/opt/conda/lib/python3.7/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
uncompiled_predictor.delete_endpoint()